In [45]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, KFold

np.set_printoptions(precision=11,suppress=True)

In [46]:
train_raw_data = pd.read_csv('train_data.csv')
test_raw_data = pd.read_csv('test_features.csv')
print(f'train rows : {len(train_raw_data)},train columns : {len(train_raw_data.columns)},test rows : {len(test_raw_data)},test columns : {len(test_raw_data.columns)}')
label=train_raw_data.loc[:,'poi']
train_raw_data = train_raw_data.drop(columns = ['poi'])

train rows : 113,train columns : 22,test rows : 33,test columns : 21


### 1e

In [47]:
pre_process_data = pd.concat([train_raw_data,test_raw_data])
pre_process_data_1 = pre_process_data.drop(columns = ['name','email_address'])
pre_process_data_1 = pre_process_data_1.fillna(0)
x_train, x_valid, y_train, y_valid = train_test_split(pre_process_data_1[:len(label)], label, test_size=0.2)
lr = LogisticRegression()
lr.fit(x_train,y_train)
y_pred = lr.predict(x_valid)
print(f'valid score logistic:{accuracy_score(y_pred,y_valid)}')
gdbtclf = GradientBoostingClassifier()
gdbtclf.fit(x_train,y_train)
y_pred = gdbtclf.predict(x_valid)
print(f'valid score of GradientBoostingClassifier:{accuracy_score(y_pred,y_valid)}')

vlid score logistic:0.782608695652174
valid score of GradientBoostingClassifier:0.9130434782608695


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [52]:
y_pred = gdbtclf.predict(pre_process_data_1[len(label):])
y_pred_prob = gdbtclf.predict_proba(pre_process_data_1[len(label):])
print(f'{y_pred}\n{y_pred_prob[:,1]}')

submission=pd.DataFrame({'name':test_raw_data['name'],'poi':pd.Series(y_pred_prob[:,1])})
submission.to_csv('submission_1e.csv',index=False)

[False  True  True False  True  True  True False False False False False
 False False  True False False False False False  True False False False
 False False False False False False False False False]
[0.0039277964  0.9449938822  0.69606032039 0.00006942354 0.60166368861
 0.98373552471 0.73894081853 0.00016465881 0.00313067771 0.00013238858
 0.00027995149 0.00006340449 0.0004779631  0.00009779309 0.94172967387
 0.00006340449 0.00070115204 0.00218753616 0.00012167782 0.00006340449
 0.63489416379 0.33923063134 0.00100288696 0.00007314518 0.00007000274
 0.00009850923 0.00004894385 0.00009343362 0.00212186588 0.00363513101
 0.00010090971 0.00016101642 0.05236129905]


---

### 2e

In [ ]:
pre_process_data = pd.concat([train_raw_data,test_raw_data])
pre_process_data_1 = pre_process_data.drop(columns = ['name','email_address'])
pre_process_data_1 = pre_process_data_1.fillna(0)